In [37]:
from metrics import *
import pandas as pd
import numpy as np
import random
from scipy.stats import multivariate_normal
import seaborn as sns

In [38]:
mall_customers_data = pd.read_csv("Mall_Customers.csv")
del mall_customers_data['CustomerID']

#print(mall_customers_data)

1. Write a function kMeans(D, k, eps, mu = None) that runs the kMeans algorithm on
a given 2D-numpy array D with k clusters and termination criterion eps. If no initial
centroids mu are given, use random points in the range of the dataset.
Let the function return a pair C, mu where C is a list that defines for each instance in
D its cluster (number). mu should be a list of k elements, containing the means of the
respective clusters.


In [39]:
def kMeans(D, k, eps, mu = None):
    rows, columns = D.shape
    
    minValues = np.min(D, axis=0)
    maxValues = np.max(D, axis=0)
    
    if mu is None:
        mu = []
        for i in range(k):
            vals = np.zeros(columns)
            for col in range(columns):
                vals[col] = (random.uniform(minValues[col], maxValues[col]))
            mu.append(vals)
    
    C = np.zeros(rows)
    
    t = 0
    while True:
        t = t+1
        C = np.zeros(rows, dtype=int)
        
        mu_last = mu.copy()
        for i, val in enumerate(D):
            minDistance = np.inf
            for k_i, centroid in enumerate(mu):
                dist = np.linalg.norm(val-centroid)
                if minDistance > dist:
                    minDistance = dist
                    C[i] = k_i+1
        
        for k_i, centroid in enumerate(mu):
            condition = (C == k_i+1)
            indexes =  np.where(condition)[0]
            
            sum_k = np.sum(D[indexes], axis=0)
            mu[k_i] = sum_k/len(indexes)
        
        conv_dist = 0
        for k_i, centroid in enumerate(mu):
            conv_dist += np.linalg.norm(mu[k_i]-mu_last[k_i])
                
        if conv_dist <= eps:
            break;

    return C, mu
    
# C, mu = kMeans(mall_customers_data._get_numeric_data().values, 3, 0.001, mu = None)
# # print(mu)
# # print(mall_customers_data._get_numeric_data().values[0:20,:])
# plotClusters(mall_customers_data._get_numeric_data().values, C, 0, 1, None, None)
# plotClusters(mall_customers_data._get_numeric_data().values, C, 0, 1, 2, None)

2 Write a function EM(D, k, eps, mu = None, independent = False) that conducts
the expectation maximization algorithm on a given 2D-numpy array D with k clusters and termination criterion eps. If no initial centroids mu are given, use random
points in the range of the dataset. If independent is set to True, treat the attributes as
independent when estimating the cluster co-variance matrices.
Let the function return a triple w, mu, cov, where w is the assignment probability matrix
(one line per point, one column per cluster), and mu, and cov are lists of means/covariance matrices of the k clusters.
Hints:

• Check that the weights for each instance sum up to 1 after the expectation step.

• If you run into numeric problems or create singular estimates of the co-variance
matrices for the clusters, you can restart your algorithm.

In [40]:
def EM(D, k, eps, mu = None, independent = False):
    rows, columns = D.shape
    
    minValues = np.min(D, axis=0)
    maxValues = np.max(D, axis=0)
    
    if mu is None:
        mu = []
        for i in range(k):
            vals = np.zeros(columns)
            for col in range(columns):
                vals[col] = (random.uniform(minValues[col], maxValues[col]))
            mu.append(vals)
    
#     x = D[:,0]
#     y = D[:,1]
#     z = D[:,2]
    
#     X =np.stack((x,y,z), axis=0)
    
    cov_M = []
    for i in range(k):
        #cov_M.append(15*np.eye(columns))
        #cov_M.append(np.tril(np.triu(np.cov(X), k=0), k=0))
        #cov_M.append(np.cov(X))
        cov_M.append(np.diag(np.var(D, axis = 0)))
        
#     variance = np.var(D, axis=0)
#     print(np.var(D, axis=0))
#     print(np.diag(variance))

#     print(np.var(D, axis = 0))
#     print(np.diag(np.var(D, axis = 0)))
        
    P_C = np.full(k, 1/k)
    t = 0
    w = np.zeros(shape=(rows, k))
    
#     print('Inicio')
#     print(mu)
    
    while True:
        t = t+1
        mu_last = mu.copy()
        for i in range(k):
            fi_xj= 0
            for j, xj in enumerate(D):
                sum_pos_prob = 0
                for a in range(k):
                    fa_xj = multivariate_normal.pdf(xj, mean=mu[a], cov=cov_M[a], allow_singular=True)
                    #fa_xj = np.nan_to_num(fa_xj)
                    sum_pos_prob += fa_xj * P_C[a]
                fi_xj = multivariate_normal.pdf(xj, mean=mu[i], cov=cov_M[i], allow_singular=True)
                wij = (fi_xj * P_C[i])/sum_pos_prob
                w[j,i] = wij 
        
        
        # Maximización
        for i in range(k):
            sum_mean = 0
            sum_cov = 0
            sum_prob = 0

            for j, xj in enumerate(D):
                sum_mean += w[j,i] * xj
        
                z = xj-mu[i]
                sum_cov += w[j,i] * (z*z.reshape(-1, 1))
                
                sum_prob += w[j,i]
                
            # Reestimate mean
            mu[i] = sum_mean/sum_prob
            
            # Reestimate covariance
            cov_M[i] = sum_cov/sum_prob
            
            if independent:
                cov_M[i] = np.tril(np.triu(cov_M[i], k=0), k=0)
            
            # Reestimate mean
            P_C[i] = sum_prob/rows
            
#         print('Iteracion ', t)
#         print(mu)
        
        conv_dist = 0
        for k_centroid, centroid in enumerate(mu):
            conv_dist += np.linalg.norm(mu[k_centroid]-mu_last[k_centroid])
        
        if conv_dist <= eps:
            break;
            
    return w, mu, cov_M

             
# w, mu, cov_M = EM(mall_customers_data._get_numeric_data().values, 2, 0.01, mu = None, independent = False)

# print(mu)
# print(cov_M)
# print(np.sum(w, axis=1))

• Write a function getClusterings(D, k) that assumes eps = 0.01 and runs the three
algorithms (EM once with dependent and once with independent attributes) for the given
k and returns a triplet C1, C2, C3 with the clusterings produced by the three algorithms
respectively (for EM, assign each point to the cluster with its highest probability).

In [41]:
def getClusterings(D, k):
    eps = 2
    
    rows, columns = D.shape
    
    # K Means
    C1, mu = kMeans(D, k, eps, mu = None)
    
    # EM Dependent
    w, mu, cov_M = EM(D, k, eps, mu = None, independent = False)
    C2 = np.zeros(rows, dtype=int)
    
    # EM Independent
    w2, mu2, cov_M2 = EM(D, k, eps, mu = None, independent = True)
    C3 = np.zeros(rows, dtype=int)
    
    for i, xi in enumerate(D):
#         max_prob = np.max(w[i])
#         index_max_prob = np.where(w[i] == max_prob)
#         print(np.argmax(w[i]))

        index_max_prob = np.argmax(w[i])
        C2[i] = index_max_prob + 1
        
        index_max_prob_2 = np.argmax(w2[i])
        C3[i] = index_max_prob_2 + 1
        
    return C1, C2, C3
    
    
C1, C2, C3 = getClusterings(mall_customers_data._get_numeric_data().values, 3)

# print(C1)
# print(C2)
# print(C3)

# plotClusters(mall_customers_data._get_numeric_data().values, C, 0, 1, None, None)
# plotClusters(mall_customers_data._get_numeric_data().values, C, 0, 1, 2, None)

# plotClusters(mall_customers_data._get_numeric_data().values, C2, 0, 1, None, None)
# plotClusters(mall_customers_data._get_numeric_data().values, C2, 0, 1, 2, None)

# plotClusters(mall_customers_data._get_numeric_data().values, C3, 0, 1, None, None)
# plotClusters(mall_customers_data._get_numeric_data().values, C3, 0, 1, 2, None)

• Write a function plotMetrics(D,l=100) that invokes getClusterings(D, k) 10 times
for k = 2, 3, 4, 5 each. The function then should create a 7 × 3 grid in which each figure
shows a boxplot. In this figure grid, each line represents a metric, and each column
represents one of the three algorithms. In plot (i, j), show the 4 box plots summarizing,
for each k, the i-th metric among the l runs for the j-th clustering algorithm.
Based on these results for l = 100, which value for k, and which algorithm would you
suggest to realize the clustering?

In [ ]:
def plotMetrics(D, l=100):
    k = [2, 3, 4, 5]
    metrics = ["beta", "cindex", "nc", "mod", "dunn", "db", "sil"]
    
    metricsData = [ [ [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ] ] ,
                    [ [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ] ] ,
                    [ [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ], [ [],[],[],[] ] ] ]
    
#     metricsData = [ [ [ [] for k_i in range(len(k)) ] for m in range(len(metrics)) ] for a in range(3)]
    
    DM = getDistances(D)
    
    for i in range(l):
        for j, kj in enumerate(k):
            # C1, C2, C3
            clusters = getClusterings(D, kj)
            for c, cluster in enumerate(clusters):
                for m, metric in enumerate(metrics):
                    mtr = getMetric(D, cluster, metric)
                    metricsData[c][m][j].append(mtr)
        print(i)

    fig, ax = plt.subplots(nrows=7, ncols=3, figsize=(15,20))
    for metric_i, row in enumerate(ax):
        for algorithm_i, col in enumerate(row):
            col.boxplot(metricsData[algorithm_i][metric_i])
    plt.show()
        
plotMetrics(mall_customers_data._get_numeric_data().values)

0


In [ ]:
#     x = range(10)
#     y = range(10)

#     fig, ax = plt.subplots(nrows=7, ncols=3, figsize=(15,20))
    
#     for row in ax:
#         for col in row:
#             col.plot(x, y)

# #     plt.show()